In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.priors import uniform_prior
from sciope.inference.abc_inference import ABC
from sciope.utilities.distancefunctions import naive_squared
from tsfresh.feature_extraction.settings import MinimalFCParameters
from sklearn.metrics import mean_absolute_error
from gillespy2.solvers.numpy import NumPySSASolver
import gillespy2
import time

/usr/lib64/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


## Define gillespy2 model

In [2]:
class ToggleSwitch(gillespy2.Model):
    """ Gardner et al. Nature (1999)
    'Construction of a genetic toggle switch in Escherichia coli'
    """
    def __init__(self, parameter_values=None):
        # Initialize the model.
        gillespy2.Model.__init__(self, name="toggle_switch")
        # Parameters
        alpha1 = gillespy2.Parameter(name='alpha1', expression=1)
        alpha2 = gillespy2.Parameter(name='alpha2', expression=1)
        beta = gillespy2.Parameter(name='beta', expression="2.0")
        gamma = gillespy2.Parameter(name='gamma', expression="2.0")
        mu = gillespy2.Parameter(name='mu', expression=1.0)
        self.add_parameter([alpha1, alpha2, beta, gamma, mu])

        # Species
        U = gillespy2.Species(name='U', initial_value=10)
        V = gillespy2.Species(name='V', initial_value=10)
        self.add_species([U, V])

        # Reactions
        cu = gillespy2.Reaction(name="r1",reactants={}, products={U:1},
                propensity_function="alpha1/(1+pow(V,beta))")
        cv = gillespy2.Reaction(name="r2",reactants={}, products={V:1},
                propensity_function="alpha2/(1+pow(U,gamma))")
        du = gillespy2.Reaction(name="r3",reactants={U:1}, products={},
                rate=mu)
        dv = gillespy2.Reaction(name="r4",reactants={V:1}, products={},
                rate=mu)
        self.add_reaction([cu,cv,du,dv])
        self.timespan(np.linspace(0,50,101))

toggle_model = ToggleSwitch()

# Define simulator function
def set_model_parameters(params, model):
    """ params - array, needs to have the same order as
        model.listOfParameters """
    for e, (pname, p) in enumerate(model.listOfParameters.items()):
        model.get_parameter(pname).set_expression(params[e])
    return model

# Here we use gillespy2 numpy solver
def simulator(params, model):

    model_update = set_model_parameters(params, model)
    num_trajectories = 1  # TODO: howto handle ensembles

    res = model_update.run(solver=NumPySSASolver, show_labels=False,
                           number_of_trajectories=num_trajectories)
    tot_res = np.asarray([x.T for x in res]) # reshape to (N, S, T)  
    tot_res = tot_res[:,1:, :] # should not contain timepoints
    
    return tot_res

# Wrapper, simulator function to abc should only take one argument (the parameter point)
def simulator2(x):
    return simulator(x, model=toggle_model)

# Set up the prior
default_param = np.array(list(toggle_model.listOfParameters.items()))[:,1] #take default from model as reference
bound = []
for exp in default_param:
    bound.append(float(exp.expression))
    
# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

In [3]:
# Generate some fixed(observed) data based on default parameters of model 
fixed_data = toggle_model.run(solver=NumPySSASolver, number_of_trajectories=100, show_labels=False)

In [4]:
# Reshape data to (n_points,n_species,n_timepoints)
fixed_data = np.asarray([x.T for x in fixed_data])
# and remove timepoints array
fixed_data = fixed_data[:,1:, :]

Sciope assumes the format of a time series to be a matrix of the form,
$N \times S \times T$,
where $N$ is the number of trajectories, $S$ is the number of species and $T$ is the number of time points or intervals.

In [5]:
# Verify the shape of the time series
fixed_data.shape

(100, 2, 101)

In case the time series are to be read from a flat file, the data will need to be in the reshaped in the $N \times S \times T$ format, if not already so.

The following cell is purely of pedagogical purpose to show writing to and reading from flat files, and data transformation into the required time series format.

In [6]:
# Writing the time series of a particular specie to a CSV flat file (if needed)
np.savetxt('specie_u.csv', fixed_data[:,0,:] , delimiter=',')

# We should have now written the time series as a matrix to the CSV file with dimensions 100 x 101

# Reading from the CSV and transforming the time series into the right format
specie_u_ts = np.loadtxt('specie_u.csv', delimiter=',')

# We now have the time series as 100 x 101 and we reshape to 100 x 1 x 101
# The '1' in the middle is necessary to signify 1 specie (U)
specie_u_ts = specie_u_ts.reshape(specie_u_ts.shape[0], 1, specie_u_ts.shape[1])
specie_u_ts.shape

(100, 1, 101)

In [7]:
# Function to generate summary statistics 
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

# Start abc instance
abc = ABC(fixed_data, sim=simulator2, prior_function=uni_prior, summaries_function=summ_func.compute, distance_function=ns)

In [8]:
# First compute the fixed (observed) mean 
abc.compute_fixed_mean(chunk_size=2)

# Run in multiprocessing mode
tic = time.time()
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)
toc = time.time() - tic
toc

1.7175843715667725

In [9]:
true_params = bound
mae_inference = mean_absolute_error(true_params, abc.results['inferred_parameters'])
mae_inference

0.15906258879502896

In [10]:
#abc.results # commented here in order not to clutter the notebook; refer to the last cell for an uncommented version

In [11]:
# Setup local cluster (dask client)
from dask.distributed import Client
c = Client()
c

Client Scheduler: tcp://127.0.0.1:38627,Cluster Workers: 4 Cores: 12 Memory: 67.35 GB


In [12]:
# Run in local cluster mode
res = abc.infer(num_samples=200, batch_size=20, chunk_size=10)

In [13]:
mae_inference = mean_absolute_error(true_params, abc.results['inferred_parameters'])
mae_inference

0.12512550110330165

In [14]:
# See results in res or abc.results
res

{'accepted_samples': [array([1.70612232, 0.99027578, 3.58539319, 3.64181778, 1.53263278]),
  array([1.17034417, 0.22878217, 0.49884638, 1.19988424, 1.70073786]),
  array([1.8068545 , 0.83188821, 3.05652376, 1.3564236 , 1.8995075 ]),
  array([1.68046819, 1.33746611, 1.54170131, 1.49196454, 1.02821921]),
  array([0.21989811, 1.73478868, 0.51840018, 1.3724985 , 1.8595483 ]),
  array([1.29496082, 0.97836898, 3.76445044, 3.1108277 , 1.70772617]),
  array([1.26305161, 1.12466473, 1.51607148, 2.55110919, 1.10796641]),
  array([0.47838179, 0.65447723, 1.58036244, 2.0504801 , 1.80658174]),
  array([1.25719088, 1.72164621, 0.23938598, 1.73408814, 1.07257702]),
  array([0.50800416, 1.09068718, 2.68719656, 2.03560458, 1.55376494]),
  array([1.82172467, 0.17714518, 3.75850048, 2.66442626, 1.33347718]),
  array([1.03763468, 1.22719873, 3.63482505, 2.95458463, 0.83360958]),
  array([0.58226789, 0.80998781, 2.3712547 , 0.40248093, 0.40855138]),
  array([1.64109598, 0.11881991, 1.01146549, 1.89086008, 